In [ ]:
!pip install -q jmd_imagescraper
!pip install -Uqq fastbook

import fastbook  #import the fast.ai library
from fastbook import *  
fastbook.setup_book()
from fastai.vision.widgets import *

#import the image scraper by @JoeDockrill, website: https://joedockrill.github.io/blog/2020/09/18/jmd-imagescraper-library/

from jmd_imagescraper.core import * 
from pathlib import Path
from jmd_imagescraper.imagecleaner import *

In [ ]:
animals=['frog','bird','dog','cat','bear','human']
path = Path().cwd()/"animals"#assigns path to current working directory

In [ ]:
duckduckgo_search(path,"frog","frogs",max_results=50)  #downloads 100 images of 'frogs' and saves it in path/cat
duckduckgo_search(path,"bird","birds",max_results=50)  #downloads 100 images of 'birds' and saves it in path/dog
duckduckgo_search(path,"dog","dogs",max_results=50)    #downloads 100 images of 'dogs' and saves it in path/dog
duckduckgo_search(path,"cat","cats",max_results=50)    #downloads 100 images of 'cats' and saves it in path/dog
duckduckgo_search(path,"bear","bears",max_results=50)  #downloads 100 images of 'bears' and saves it in path/dog
duckduckgo_search(path,"human","humans",max_results=50)#downloads 100 images of 'humans' and saves it in path/dog

In [ ]:
display_image_cleaner(path)

In [ ]:
fns=get_image_files(path)
fns

In [ ]:
failed=verify_images(fns)#looks for files that arent images
failed

In [ ]:
path.ls


In [ ]:
animals=DataBlock(
  blocks=(ImageBlock,CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2,seed=42),#splits the dataset into a training set and validation set
    get_y=parent_label,
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms())

In [ ]:
dls=animals.dataloaders(path)

In [ ]:
dls.valid.show_batch(max_n=6,nrows=3)

In [ ]:
learn = cnn_learner(dls, resnet18, metrics=error_rate)#training our model
learn.fine_tune(3)

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix()#plots a confusion matrix

In [ ]:
interp.plot_top_losses(20, nrows=4)

In [ ]:
cleaner=ImageClassifierCleaner(learn)
cleaner

In [ ]:
# for idx in cleaner.delete(): cleaner.fns[idx].unlink()
# for idx,cat in cleaner.change(): shutil.move(str(cleaner.fns[idx]), path/cwd) 

In [ ]:
learn.export()
path = Path()
path.ls(file_exts='.pkl')

learn_inf = load_learner(path/'export.pkl')

learn_inf.dls.vocab

In [ ]:
btn_upload = widgets.FileUpload()#Displays a widget which can be used to upload images from your system
btn_upload

In [ ]:
img = PILImage.create(btn_upload.data[-1])

out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(128,128))#resizes the uploaded image
out_pl

In [ ]:
pred,pred_idx,probs = learn_inf.predict(img)#asks the model to predict the animal in the image!
lbl_pred = widgets.Label()
lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'#displays the output in a readable format
lbl_pred

In [ ]:
btn_run = widgets.Button(description='Classify')
btn_run

In [ ]:
def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(128,128))
    pred,pred_idx,probs = learn_inf.predict(img)
    lbl_pred.value = f'Prediction: {pred}; Probability: {probs[pred_idx]:.04f}'

btn_run.on_click(on_click_classify)

In [ ]:
btn_upload=widgets.FileUpload()

In [ ]:
#hide_output
VBox([widgets.Label('Select an Image'), 
      btn_upload, btn_run, out_pl, lbl_pred])